In [2]:
import numpy as np
import pandas as pd
import arabic
import pyarabic
import helpers
from helpers import Clean_data,separate_token_with_dicrites,save_h5
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from numpy import array, argmax
import os,errno,re,sys
import pickle
from itertools import product 
from pyarabic.araby import strip_tashkeel, strip_tatweel
import h5py
import random as rn
##########################
rn.seed(123456)
np.random.seed(123456)

In [3]:
'''
    * Paths
'''
all_data_path = "../../data/Almoso3a_Alshe3rya/data/All_ksaied.csv"

'''
    * Creating Directories which will contain the Encoded data.
'''
try:
    os.makedirs("../../data/Encoded/8bits/WithoutTashkeel/Eliminated/")
    os.makedirs("../../data/Encoded/8bits/WithoutTashkeel/Full_Data/")
    os.makedirs("../../data/Encoded/8bits/WithTashkeel/Eliminated/")
    os.makedirs("../../data/Encoded/8bits/WithTashkeel/Full_Data/")
except OSError as e:
    if e.errno != errno.EEXIST:
        print("Can't create file for checkpoints or for logs please check ")
        raise
print("Paths created")

Paths created


In [4]:
'''
    * Load and Clear
'''
all_data = pd.read_csv(all_data_path, index_col=0)
print( len(all_data) )

/usr/local/lib/python3.5/dist-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


1831770


In [5]:
'''
    * (X,Y) form of the dataset. [uncleaned]
'''
all_data.info(memory_usage='deep')
bahr = 'Bahr'
bayt = 'Bayt_Text'
#all_data  = all_data[[bayt, bahr]]
bahr = 'البحر'
bayt = 'البيت'
all_data  = all_data[[bayt, bahr]]
all_data.columns = ['Bayt_Text','Bahr']

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1831770 entries, 0 to 1835252
Data columns (total 8 columns):
العصر           object
الشاعر          object
الديوان         object
القافية         object
البحر           object
الشطر الايسر    object
الشطر الايمن    object
البيت           object
dtypes: object(8)
memory usage: 1.8 GB


In [ ]:
'''
    * Cleaning Data
    # Outputs:
        $ all_data_cleaned
        % Cleaned, and Ready for filtering, encoding
'''

# Computing the maximum bayt length
MAX_LEN_BAYT = np.max((all_data.Bayt_Text.apply(pyarabic.araby.strip_tashkeel).apply(len)))


# Cleaning 
all_data_cleaned = Clean_data(data_frame=all_data,
                              max_bayt_len=MAX_LEN_BAYT,
                              verse_column_name='Bayt_Text')

15
46
92
140
213
247
269
338
341
352
353
356
366
414
444
483
534
548
576
665
722
786
802
828
829
841
879
909
956
975
979
1000
1061
1070
1092
1111
1123
1154
1213
1248
1276
1311
1327
1350
1357
1390
1407
1417
1428
1433
1479
1509
1559
1619
1656
1702
1708
1710
1713
1791
1823
1830
1862
1871
1879
1891
1911
1953
1966
1970
1983
1984
1998
2054
2100
2107
2117
2137
2179
2185
2198
2295
2314
2315
2382
2397
2441
2466
2476
2477
2532
2589
2630
2678
2679
2684
2685
2741
2825
2878
2949
2998
2999
3008
3010
3036
3143
3188
3197
3240
3366
3368
3420
3422
3477
3501
3550
3572
3601
3614
3622
3662
3665
3679
3694
3727
3746
3816
3818
3820
3832
3854
3858
3928
3981
3989
3992
4083
4087
4097
4098
4116
4125
4129
4135
4144
4194
4213
4216
4223
4241
4300
4374
4384
4474
4545
4585
4586
4696
4701
4704
4755
4767
4775
4857
4858
4861
4867
4874
4886
4895
4921
4931
5020
5046
5072
5105
5137
5139
5158
5159
5172
5216
5292
5415
5433
5510
5515
5522
5534
5535
5538
5542
5561
5593
5601
5603
5615
5633
5656
5738
5809
5853
5866
5898
5943
5954

In [22]:
'''
    * Filtering the data
    # Outputs:
        1 all_filtered_data
        2 eliminated_filtered_data
'''

elminated_classic_bohor = ['الوافر', 'المنسرح','المجتث', 'المتقارب', 'الكامل', 'الطويل', 'السريع', 'الرمل',
                         'الرجز', 'الخفيف', 'البسيط']

full_bohor_classes = ['الوافر', 'المنسرح', 'المديد', 'المجتث', 'المتقارب', 'الكامل', 'الطويل', 'السريع', 'الرمل',
                         'الرجز', 'الخفيف', 'البسيط', 'المقتضب', 'الهزج', 'المضارع', 'المتدارك']

print(elminated_classic_bohor)
print(full_bohor_classes)

all_filtered_data = all_data_cleaned.loc[all_data_cleaned['Bahr'].isin(full_bohor_classes)]
eliminated_filtered_data = all_data_cleaned.loc[all_data_cleaned['Bahr'].isin(elminated_classic_bohor)]

#all_filtered_data.head().head()
#eliminated_filtered_data.head()
#full_filtered_data.iloc[:, 1].value_counts()
#eliminated_filtered_data.iloc[:, 1].value_counts()

['الوافر', 'المنسرح', 'المجتث', 'المتقارب', 'الكامل', 'الطويل', 'السريع', 'الرمل', 'الرجز', 'الخفيف', 'البسيط']
['الوافر', 'المنسرح', 'المديد', 'المجتث', 'المتقارب', 'الكامل', 'الطويل', 'السريع', 'الرمل', 'الرجز', 'الخفيف', 'البسيط', 'المقتضب', 'الهزج', 'المضارع', 'المتدارك']


# Let's Encode <3

In [ ]:
'''
    Taha; Encoding Full Data classes as OneHot vectors. 
'''

full_data_label_encoder = LabelEncoder()
full_data_integer_encoded = full_data_label_encoder.fit_transform(list(full_filtered_data["Bahr"]))
# binary encode
full_data_onehot_encoder = OneHotEncoder(sparse=False)
full_data_integer_encoded = full_data_integer_encoded.reshape(len(full_data_integer_encoded), 1)
full_data_bohor_encoded = full_data_onehot_encoder.fit_transform(full_data_integer_encoded)
save_h5('../../data/Encoded/8bits/WithoutTashkeel/Full_Data/full_data_Y_Meters.h5', 'Y', full_data_bohor_encoded)
save_h5('../../data/Encoded/8bits/WithTashkeel/Full_Data/full_data_Y_Meters.h5', 'Y', full_data_bohor_encoded)
#saving with pickle: https://stackoverflow.com/questions/11218477/how-can-i-use-pickle-to-save-a-dict
print('saving preprocessors....')
with open("../../encoders_full_dat.pickle", 'wb') as pre_file:
    pickle.dump(full_data_label_encoder, pre_file, protocol=pickle.HIGHEST_PROTOCOL)
print('saved ....')


'''
    Taha; This cell is entirly doublicated.
    It Encodes the eliminated data classes.
'''
eliminated_data_label_encoder = LabelEncoder()
eliminated_data_integer_encoded = eliminated_data_label_encoder.fit_transform(list(eliminated_filtered_data["Bahr"]))
# binary encode
eliminated_data_onehot_encoder = OneHotEncoder(sparse=False)
eliminated_data_integer_encoded = eliminated_data_integer_encoded.reshape(len(eliminated_data_integer_encoded), 1)
eliminated_data_bohor_encoded = eliminated_data_onehot_encoder.fit_transform(eliminated_data_integer_encoded)
save_h5('../../data/Encoded/8bits/WithoutTashkeel/Eliminated/Eliminated_data_Y_Meters.h5', 'Y', eliminated_data_bohor_encoded)
save_h5('../../data/Encoded/8bits/WithTashkeel/Eliminated/Eliminated_data_Y_Meters.h5', 'Y', eliminated_data_bohor_encoded)
#saving with pickle: https://stackoverflow.com/questions/11218477/how-can-i-use-pickle-to-save-a-dict
print('saving preprocessors....')
with open("../../encoders_eliminated_data.pickle", 'wb') as pre_file:
    pickle.dump(eliminated_data_bohor_encoded, pre_file, protocol=pickle.HIGHEST_PROTOCOL)
print('saved ...')

In [ ]:
'''
    * Encoding the full data with Tashkeel
'''


X = all_filtered_data.Bayt_Text.apply(lambda x: helpers.string_with_tashkeel_vectorizer(x, MAX_LEN_BAYT))
print("full_cleaned_data_with_tashkeel_encoded encoded ..")

X_stacked = np.stack(X,axis=0)
print("stacked!")

save_h5('../../data/Encoded/8bits/WithTashkeel/Full_Data/full_data_matrix_with_tashkeel_8bitsEncoding.h5', 'X', X_stacked) 
print("full_cleaned_data_with_tashkeel_encoded saved ..")
print(X_stacked.shape)

In [ ]:
'''
    * Encoding the eleiminated data with Tashkeel
'''

X = eliminated_filtered_data.Bayt_Text.apply(lambda x: helpers.string_with_tashkeel_vectorizer(x, MAX_LEN_BAYT))
print("eliminated_cleaned_data_with_tashkeel_encoded encoded ..")

X_stacked = np.stack(X,axis=0)
save_h5('../../data/Encoded/8bits/WithTashkeel/Eliminated/eliminated_data_matrix_with_tashkeel_8bitsEncoding.h5', 'X', X_stacked)

print("full_cleaned_data_with_tashkeel_encoded saved ..")
print(X_stacked.shape)

In [ ]:
# Stripping Taskeel
all_data_cleaned = all_data_cleaned.apply(pyarabic.araby.strip_tashkeel)
all_filtered_data = all_data_cleaned.loc[all_data_cleaned['Bahr'].isin(full_bohor_classes)]
eliminated_filtered_data = all_data_cleaned.loc[all_data_cleaned['Bahr'].isin(elminated_classic_bohor)]

In [ ]:
'''
    * Encoding all data without Tashkeel
'''
X = all_filtered_data.apply(lambda x: helpers.string_with_tashkeel_vectorizer(x, MAX_LEN_BAYT))
print("full_filtered_data_without_tashkeel encoded ..")

X_stacked = np.stack(X, axis=0)
save_h5('../../data/Encoded/8bits/WithoutTashkeel/Full_Data/full_data_matrix_without_tashkeel_8bitsEncoding.h5', 'X', X_stacked) 
print("full_filtered_data_without_tashkeel_staked saved ..")

print(full_filtered_data_without_tashkeel_staked.shape)

In [ ]:
'''
    * Encoding all data without Tashkeel
'''
X = eliminated_filtered_data_without_tashkeel.apply(lambda x: helpers.string_with_tashkeel_vectorizer(x, MAX_LEN_BAYT))
print("eliminated_filtered_data_without_tashkeel encoded ..")

X_stacked = np.stack(X, axis=0)
save_h5('../../data/Encoded/8bits/WithoutTashkeel/Eliminated/eliminated_data_matrix_without_tashkeel_8bitsEncoding.h5', 'X', X_stacked) 
print("eliminated_filtered_data_without_tashkeel_staked saved ..")

print(eliminated_filtered_data_without_tashkeel_staked.shape)